# Group assignment
1. Regroup according to your group number and select the corresponding parameter sets (in DAY5/parameters for group.xlsx)

2. Copy the notebook 9. Fill out the parameters marked with a ‘??’ with your values and put the resulting plots in a PowerPoint.

3. Gather according to the soil Van Genuchten parameters and the boundary conditions.

4. Discuss the effect of the plant and soil parameters on the outputs of the 1D simulation. E.g.: 

    4.1. Do we have water stress? If yes, when? How do the effective soil water content ?

    4.2. How do hydraulic and architectural traits affect stress level? And soil water storage?  

    4.3. Can you imagine (and test) ideal traits that would minimize the stress?

5. Use the plots to prepare a short presentation (max 10mn)

In [ ]:
import os
#sourcedir = os.getcwd()+"/../../../"
sourcedir = "/opt/dumux/CPlantBox/"
import sys;  
sys.path.append(sourcedir+"../dumux-rosi/python/modules");
sys.path.append(sourcedir+"../dumux-rosi/build-cmake/cpp/python_binding/");
sys.path.append(sourcedir)  
sys.path.append(sourcedir+"src") 

import plantbox as pb
import visualisation.vtk_plot as vp
from functional.PlantHydraulicParameters import PlantHydraulicParameters
from functional.PlantHydraulicModel import HydraulicModel_Doussan
from functional.PlantHydraulicModel import HydraulicModel_Meunier
from functional.Perirhizal import PerirhizalPython as Perirhizal
import functional.van_genuchten as vg

from rosi_richards import RichardsSP  # C++ part (Dumux binding)
from richards import RichardsWrapper  # Python part
import numpy as np
import matplotlib.pyplot as plt
import timeit

def sinusoidal(t):
    return np.sin(2. * np.pi * np.array(t) - 0.5 * np.pi) + 1.


def make_source(q, area):
    s = {}
    for i in range(0, len(q)):
        if not np.isnan(q[i]):
            s[i] = -q[i] * area

    return s

In [14]:
""" Parameters """  
verbose = False

depth = 100    
min_b = np.array([-38., -8., -depth])
max_b = np.array([38., 8., 0.])
cell_number = [1, 1,depth]  # [cm3]

path = sourcedir + "modelparameter/structural/rootsystem/"
name = "Zeamays_synMRI_modified"  #"Anagallis_femina_Leitner_2010"  # Zea_mays_1_Leitner_2010, Zeamays_synMRI.xml  <<<<-------

trans_ = ??
trans = (max_b[0] - min_b[0]) * (max_b[1] - min_b[1]) * trans_  # cm3 / day
wilting_point = -15000  # cm
rs_age = 8 * 7  # 56 days  # root system initial age [day]
sim_time = 7.5  # [day]
dt = 3600. / (24 * 3600)  # [days] 

# Define van Genuchten parameters for sand, loam and clay  
# theta_r (-), theta_s (-), alpha (1/cm), n (-), Ks (cm d-1)
soil_params = ??
sp = vg.Parameters(soil_params)  # needed for Perirhizal class
vg.create_mfp_lookup(sp, wilting_point = -15000, n = 1501)  # needed for Perirhizal class


""" Initialize macroscopic soil model """
s = RichardsWrapper(RichardsSP())  
s.initialize()
s.createGrid(min_b, max_b, cell_number, periodic = True)  # [cm]

# initial soil water potential
s.setHomogeneousIC(??, False)  # [cm] False = matrix, True, = total potential

# upper and lower boundary conditions
s.setTopBC(??) 
s.setBotBC(??) 

s.setVGParameters([soil_params])
s.setParameter("Soil.SourceSlope", "100") 
s.initializeProblem()
s.setCriticalPressure(wilting_point)  

""" Initialize xylem model """
plant = pb.MappedPlant() 
plant.enableExtraNode()

# plant structural parameters
plant.readParameters(path + name + ".xml")
for p in plant.getOrganRandomParameter(pb.root):
    if (p.subType == 1): # tap root
        p.r = ??
        p.ln = ??
        p.lmax = ??
        p.tropismS = ??
        pass
    if (p.subType == 4): # basal root
        p.r = ??
        p.ln = ??
        p.lmax = ??
        p.tropismS = ??
        p.theta = ??
    if (p.subType == 5): # shoot born root
        p.r = ??
        p.ln = ??
        p.lmax = ??


sdf = pb.SDF_PlantBox(np.inf, np.inf, max_b[2] - min_b[2] - 0.5)  
plant.setGeometry(sdf)  
plant.setRectangularGrid(pb.Vector3d(min_b), pb.Vector3d(max_b), pb.Vector3d(cell_number), False, False)  # needed for Perirhizal class

""" root hydraulic properties """
# axial and radial conductivities
param = PlantHydraulicParameters()  
kr_main_roots_max = ??
kx_main_roots_max = ??
kr_lat_roots_max = ??
kx_lat_roots_max = ??

kr_main_roots_age = np.array([0., 12.5, 20.9, 44.6, 62.7, 100])
kr_main_roots_val = np.array([kr_main_roots_max, kr_main_roots_max, 
                              kr_main_roots_max/2, kr_main_roots_max/2, 
                              kr_main_roots_max/10, kr_main_roots_max/10])

kr_lat_roots_age = np.array([0., 10, 15, 25])
kr_lat_roots_val = np.array([kr_lat_roots_max, kr_lat_roots_max, 
                             kr_lat_roots_max/10, kr_lat_roots_max/10])
param.set_kr_age_dependent(kr_main_roots_age, kr_main_roots_val, subType = [1, 4, 5])  
param.set_kr_age_dependent(kr_lat_roots_age, kr_lat_roots_val, subType = [2, 3])


kx_main_roots_age = np.array([0., 18.3, 21, 47, 61, 100])
kx_main_roots_val = np.array([kx_main_roots_max/200, kx_main_roots_max/200, 
                              kx_main_roots_max/10, kx_main_roots_max/10, 
                              kx_main_roots_max, kx_main_roots_max])

kx_lat_roots_age = np.array([0., 9, 13, 20, 25])
kx_lat_roots_val = np.array([kx_lat_roots_max/200, kx_lat_roots_max/100, 
                             kx_lat_roots_max/4, kx_lat_roots_max, kx_lat_roots_max])

param.set_kx_age_dependent(kx_main_roots_age, kx_main_roots_val, subType = [1, 4, 5])
param.set_kx_age_dependent(kx_lat_roots_age, kx_lat_roots_val, subType = [2, 3])

hm = HydraulicModel_Doussan(plant, param)
hm.wilting_point = wilting_point  

""" Coupling (map indices) """
picker = lambda x, y, z: s.pick([0, 0, z]) 
plant.setSoilGrid(picker)

plant.initialize(True)
plant.simulate(rs_age, True)
hm.test() 

peri = Perirhizal(plant)
h_bs = s.getSolutionHead()
h_sr = np.ones(h_bs.shape) * wilting_point

""" Numerical solution """
start_time = timeit.default_timer()
t = 0.
time_, tact_, h_eff_, water_volume_ = [], [], [], []
N = round(sim_time / dt)
area = (plant.maxBound.x - plant.minBound.x) * (plant.maxBound.y - plant.minBound.y)  # [cm2]

for i in range(0, N): 

    h_bs = s.getSolutionHead()
    h_bs = np.array(plant.matric2total(h_bs))

    start_time_ao = timeit.default_timer()

    hm.update(rs_age + t)

    # Alpha: root system averaged stress factor
    # krs, _ = hm.get_krs(rs_age + sim_time)  # [cm2/day] (could be precomputed for static case)
    krs = hm.krs
    krs = krs / area

    k_srs = hm.get_soil_rootsystem_conductance(rs_age + sim_time, h_bs, wilting_point, sp)
    h_bs_diff = h_bs - np.ones(h_bs.shape) * wilting_point
    alpha = np.multiply(k_srs, h_bs_diff) / (-krs * wilting_point)  # [1]

    # Omega: root system averaged stress factor
    # suf_ = hm.get_suf(rs_age + sim_time)
    suf_ = hm.suf
    suf = peri.aggregate(suf_[0,:])
    alphaSUF = np.multiply(alpha, suf)
    omega = np.nansum(alphaSUF)  # note that nan are treated as 0

    # Omega_c: critical stress factor
    tp = trans * sinusoidal(t) / area  # potential tranpiration [cm3 day-1] -> [cm day-1]
    
    omega_c = tp / (-wilting_point * krs)

    # Sink, stressed
    q_s = alphaSUF * tp / omega_c

    # Sink, unstressed
    denumerator = np.multiply(h_bs_diff, np.nansum(np.divide(alphaSUF, h_bs_diff)))
    q_us = alphaSUF * tp / omega_c - np.divide(alphaSUF, denumerator) * (omega / omega_c - 1) * tp

    
    if omega < omega_c:
        q = q_s
    else:
        q = q_us

    start_time_soil = timeit.default_timer()

    fluxes = make_source(q, area)
    s.setSource(fluxes)
    s.solve(dt) 

    
    water_volume = s.getWaterVolume() 


    water_volume_.append(water_volume)
    h_eff_.append(sum(suf*h_bs))
    x_.append(t)
    tact_.append(-np.nansum(q) * area) 

    n = round(float(i) / float(N) * 100.)  
    print("[" + ''.join(["*"]) * n + ''.join([" "]) * (100 - n) + "], potential {:g}, actual {:g}; [{:g}, {:g}] cm soil at {:g} days"
            .format(tp * area, np.nansum(q) * area, np.min(h_bs), np.max(h_bs), s.simTime))

    t += dt  # [day]

print ("Coupled benchmark solved in ", timeit.default_timer() - start_time, " s")  # |\label{l7xa:timing}|

""" Get plots """
fig, ax1 = plt.subplots()
ax1.plot(time_, trans * sinusoidal(time_), 'k')  # potential transpiration
ax1.plot(time_, -np.array(tact_), 'g')  # actual transpiration
ax2 = ax1.twinx()
ax2.plot(time_, np.cumsum(-np.array(tact_) * dt), 'c--')  # cumulative transpiratio
ax1.set_xlabel("Time [d]")
ax1.set_ylabel("Transpiration $[mL d^{-1}]$ per plant")
ax1.legend(['Potential', 'Actual', 'Cumulative'], loc = 'upper left')
#plt.show()
fig.savefig("results/transpiration.jpg")
plt.close(fig)


vp.plot_roots_and_soil(hm.ms.mappedSegments(), "pressure head", hm.get_hs(h_bs), s, False, min_b, max_b, 
                       cell_number, 
                       interactiveImage = False)  # VTK vizualisation

depth_array = [cc[2] for cc in s.getCellCenters()]
fig, ax1 = plt.subplots()
ax1.plot(q/np.nansum(q), depth_array,label = "RWU fraction [-]", linewidth=4 )  
ax1.plot(suf, depth_array, label = "SUF [-]")
ax2 = ax1.twiny()
ax2.plot(h_bs, depth_array, label="Soil water potential [cm]",color='g') 
ax2.set_xlabel("Water potential [cm]") 
ax1.set_xlabel("Fraction [-]")
ax1.set_ylabel("Depth [cm]")
ax1.legend(bbox_to_anchor=(1.7, 0.75))
ax2.legend(bbox_to_anchor=(1.7, 0.5))
#plt.show()
fig.savefig("results/suf.jpg", bbox_inches="tight")
plt.close(fig)

    
fig, ax1 = plt.subplots() #water_volume_
ax1.plot(time_, h_eff_, 'k')  
#ax1.set_ylabel("Water potential $[cm]$")
ax1.set_title("Effective soil water potential [cm]") 
ax1.set_xlabel("Time $[d]$")
ax1.legend(bbox_to_anchor=(1.7, 0.5))
#plt.show()
fig.savefig("results/effective_soil_potential.jpg")
plt.close(fig)

fig, ax1 = plt.subplots() #water_volume_
ax1.plot(time_, water_volume_, 'k')  
ax1.set_xlabel("Time $[d]$")
ax1.legend(bbox_to_anchor=(1.7, 0.5))
ax1.set_title("Soil water volume [cm3 water]")  
fig.savefig("results/water_volume.jpg")
#plt.show()
plt.close(fig)
